<a href="https://colab.research.google.com/github/jagatabhay/DANN/blob/master/S4_Assignmet_Acc99_32.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        #self.conv1 = nn.Conv2d(1, 32, 3, padding=1) #input -? OUtput? RF
        #self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        #self.pool1 = nn.MaxPool2d(2, 2)
        #self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        #self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        #self.pool2 = nn.MaxPool2d(2, 2)
        #self.conv5 = nn.Conv2d(256, 512, 3)
        #self.conv6 = nn.Conv2d(512, 1024, 3)
        #self.conv7 = nn.Conv2d(1024, 10, 3)

        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)
        #self.bat2  = nn.BatchNorm2d(1)
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)
        self.pool1 = nn.MaxPool2d(2, 2)
        #self.conv3 = nn.Conv2d(16, 16, 3, padding=1)
        self.bat1  = nn.BatchNorm2d(16)
        self.conv4 = nn.Conv2d(16, 32, 3, padding=1)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.bat3  = nn.BatchNorm2d(32)
        self.conv5 = nn.Conv2d(32 ,8 , 1 )
        
        self.conv6 = nn.Conv2d(32, 16, 3)
        self.conv7 = nn.Conv2d(16, 32, 3)
        self.conv8 = nn.Conv2d(32, 10, 3)
        #self.conv9 = nn.Conv2d(64, 10, 3)


    def forward(self, x):
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        x = self.pool2(F.relu(self.conv4(F.relu(self.bat1(x)))))
        x = F.relu(self.conv6(F.relu(self.bat3(x))))
        x = self.conv8(F.relu(self.conv7(x)))
        #x = F.relu(self.conv8(x))
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [3]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
            Conv2d-2           [-1, 16, 28, 28]           1,168
         MaxPool2d-3           [-1, 16, 14, 14]               0
       BatchNorm2d-4           [-1, 16, 14, 14]              32
            Conv2d-5           [-1, 32, 14, 14]           4,640
         MaxPool2d-6             [-1, 32, 7, 7]               0
       BatchNorm2d-7             [-1, 32, 7, 7]              64
            Conv2d-8             [-1, 16, 5, 5]           4,624
            Conv2d-9             [-1, 32, 3, 3]           4,640
           Conv2d-10             [-1, 10, 1, 1]           2,890
Total params: 18,138
Trainable params: 18,138
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 0.27
Params size (MB): 0.07
Estimated Tot

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:38: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [4]:


torch.manual_seed(1)
batch_size = 30

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw


Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...



Done!


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 10):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0402, Accuracy: 9868/10000 (98.68%)



  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0410, Accuracy: 9862/10000 (98.62%)



  0%|          | 0/2000 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0349, Accuracy: 9912/10000 (99.12%)



  0%|          | 0/2000 [00:00<?, ?it/s]


Test set: Average loss: 0.0267, Accuracy: 9915/10000 (99.15%)



loss=9.253819735022262e-05 batch_id=1999: 100%|██████████| 2000/2000 [00:28<00:00, 69.61it/s]



Test set: Average loss: 0.0318, Accuracy: 9914/10000 (99.14%)

